In [34]:
import pandas as pd
from pathlib import Path

# Vorverarbeitung des Datensatzes

In [35]:
input_data_folder = "../data/"
input_data_filename = "AAB_cleaned.csv"
output_data_filename = "AAB_cleaned.csv"

raw_data_file = Path(input_data_folder) / input_data_filename
cleaned_data_file = Path(input_data_folder) / input_data_filename

In [36]:
df_cars = pd.read_csv(raw_data_file, sep= "\t")  

# Variante für Google Colab
#df_cars = pd.read_csv( "https://raw.githubusercontent.com/SebastianZug/RoboLabVortraege/refs/heads/main/30_PythonAutomobile/project/data/AAB_cleaned.csv", sep= "\t")  

In [37]:
df_cars.head()

,Seite,Familienname,Vorname,Kennzeichen,Nummer,Beruf_Stand,Wohnort,Adresse,Fzg_Typ,Kreis_Stadt,Firma,id
0,103,Trautmann,Robert,S S,80,Privatmann,Arnstadt,Lindenallee 11,Lxw.,Arnstadt,-,17091280
1,420,Beckmann,Otto,I K,15,Molk.-Techniker,Breslau,N. Taschenstr. 25 a,KrR.,Provinz Schlesien,-,17101628
2,429,Barteczko,Karl,I K,439,Maschinist,Ratibor,Neuer Markt 5,KrR.,Provinz Schlesien,-,17101933
3,529,"Leesen, v.",-,I P,233,Ingenieur,St. Margarethen,-,KrR.,Schleswig-Holstein,-,17105384
4,2,-,-,A,73,-,Stadt Dessau,Franzstr. 30,LW.,Stadt Dessau,Berlin-Anh. Maschinenbau-Aktien-Gesellschaft,17088030


## Schritt 1: Korrektur individueller Typos

In [38]:
# Beispiel "Hermann-Fehler" offenbar wurde hier ohne Kenntnis der Bedeutung aus der Abkürzung von Herrmann ein 
# Herrn. gemacht
df_cars[df_cars.Vorname=="Herrn."].shape

(43, 12)

In [39]:
df_cars[df_cars.Vorname=="Herrn."].head(10)

,Seite,Familienname,Vorname,Kennzeichen,Nummer,Beruf_Stand,Wohnort,Adresse,Fzg_Typ,Kreis_Stadt,Firma,id
852,168,Käbisch,Herrn.,III,207,Flußaufseher,Leipzig-Connewitz,-,KrR.,Kreishauptmannschaft Leipzig,-,17093104
3210,734,Breuer,Herrn.,I Z,581,Bäckermeister,Duisburg,Wilhelmstr. 22,GW.,Rheinprovinz,-,17112221
3332,976,Scherer,Herrn.,II U,74,K. Intendanturrat,Würzburg,Ludwigkai 9,Lxw.,Regierungsbezirk Unterfranken,-,17120522
4312,1201,Wagner,Herrn.,VI C,721,Büchsenmacher,Ban St. Martin,-,KrR.,Kreis Metz,-,17127282
4736,34,Bode,Herrn.,H H,187,Schneider,Hamburg,Mathildenstraße 1 III.,KrR.,Hamburg,-,17089071
7397,662,Steinmann,Herrn.,I X,170,Fuhrunternehm.,Bottrop,-,KrR.,Provinz Westfalen,-,17109848
9454,481,Wohlsdorf,Herrn.,I M,127,Wagenmstr.,Osterwieck a. H.,Wilhelm­,KrR.,Provinz Sachsen,-,17103724
9711,438,Jüttner,Herrn.,I K,957,Bauunternehmer,Liegnitz,Neue Haynauer Straße 4,KrR.,Provinz Schlesien,-,17102257
10133,690,Harkort,Herrn.,I X,2774,Fabrikbesitzer,Haus Schede b. Herdecke,-,Lxw.,Provinz Westfalen,-,17110767
10160,773,Peill,Herrn.,I Z,3480,Landwirt,Cöln,Rheinaustr. 38,KrR.,Rheinprovinz,-,17113627


Überprüfung der Daten

[https://des.genealogy.net/search/show/17110767](https://des.genealogy.net/search/show/17110767)

In [52]:
typos = {
    r"\.\.": ".",                   #35005
    r"^ß": "B",                     #34813 ßruno -> Bruno 
    r"Wilhelm ll": "Wilhelm II",    #3792  auf Seite 1006 geprüft
    r"Robert\.": "Robert",          #29230, 44451 auf Seite 1128, 142 geprüft
    r"rob.": "Robert",              #35184 auf Seite 203 geprüft
    r"Wilm\.": "Wilh."  ,           #29156 auf Seite 1118 geprüft
    r"Wilhelm\.": "Wilhelm",        #26858, 31482 auf Seite 165, 317 geprüft
    r"Albert\.": "Albert",          #41203 auf Seite 203 geprüft
    r"J ohs\.": "Johs.",            #34241 auf Seite 555 geprüft
    r"J oh\.": "Joh.", 
    r"wilh\.": "Wilh.",    
    r"Freidr.": "Friedr.",
    r"August\.": "August",          #35081, 41268
    r"Emil\.": "Emil",              #44570 auf Seite 112 geprüft
    r"Dieter\.": "Dieter",          #30410
    r"Fricdr\.": "Friedr.",         #894   auf Seite 173 geprüft
    r"Karl\.": "Karl",              #31480
    r"Herrn\.": "Herm."             #Fehler kommt sehr häufig vor 
}

df_cars['_Vorname'] = df_cars['Vorname']
df_cars['_Vorname_Adaption'] = False
for typo in typos:
    relevant_df = df_cars.Vorname.str.contains(typo, na=False, regex=True)
    print(f"Typo: {typo} - {relevant_df.sum()} Treffer")
    df_cars.loc[relevant_df, "_Vorname"] = df_cars.loc[relevant_df, "_Vorname"].str.replace(typo, typos[typo], regex=True)
    df_cars.loc[relevant_df, "_Vorname_Adaption"] = True

Typo: \.\. - 1 Treffer
Typo: ^ß - 1 Treffer
Typo: Wilhelm ll - 1 Treffer
Typo: Robert\. - 2 Treffer
Typo: rob. - 1 Treffer
Typo: Wilm\. - 1 Treffer
Typo: Wilhelm\. - 2 Treffer
Typo: Albert\. - 1 Treffer
Typo: J ohs\. - 1 Treffer
Typo: J oh\. - 1 Treffer
Typo: wilh\. - 1 Treffer
Typo: Freidr. - 1 Treffer
Typo: August\. - 2 Treffer
Typo: Emil\. - 1 Treffer
Typo: Dieter\. - 1 Treffer
Typo: Fricdr\. - 1 Treffer
Typo: Karl\. - 1 Treffer
Typo: Herrn\. - 49 Treffer


In [53]:
# find all names that were adapted
df_cars[df_cars._Vorname_Adaption][["Familienname", "Vorname", "_Vorname"]].head(10)

,Familienname,Vorname,_Vorname
852,Käbisch,Herrn.,Herm.
894,Schroth,Fricdr. Herm.,Friedr. Herm.
2223,Riemenschneider,Freidr.,Friedr.
3210,Breuer,Herrn.,Herm.
3332,Scherer,Herrn.,Herm.
3792,Seine Majestät der König,Wilhelm ll.,Wilhelm II.
4312,Wagner,Herrn.,Herm.
4736,Bode,Herrn.,Herm.
7397,Steinmann,Herrn.,Herm.
9454,Wohlsdorf,Herrn.,Herm.


## Schritt 2: Korrektur von Abkürzungen

In [ ]:
most_used_abrevations={
    'Ad.': 'Adolf', 
    'Alb.': 'Albert', 
    'Alex.': 'Alexander', 
    'Alfr.': 'Alfred', 
    'Ant.': 'Anton', 
    'Aug.': 'August', 
    'Benj.': 'Benjamin', 
    'Bernh.': 'Bernhard', 
    'Charl.': 'Charles', 
    'Chr.': 'Christian', 
    'Christ.': 'Christian', 
    'Detl.': 'Detlef', 
    'Dietr.': 'Dieter', 
    'Eberh.': 'Eberhard', 
    'Edm.': 'Edmund', 
    'Em.': 'Emanuel', 
    'Engelb.': 'Engelbrecht',
    'Engelbr.': 'Engelbrecht', 
    'Erh.': 'Ehrhard',
    'Eug.': 'Eugen', 
    'Ferd.': 'Ferdinand', 
    'Ferdin.': 'Ferdinand', 
    'Fr.': "Friedrich",
    'Frdr.': 'Friedrich', 
    'Frhr.': 'Freiherr', 
    'Fried.': 'Friedrich', 
    'Friedr.': 'Friedrich', 
    'Frz.': 'Franz', 
    'Gerh.': 'Gerhard', 
    'Gg.': 'Georg', 
    'Gottf.': 'Gottfried', 
    'Gottfr.': 'Gottfried', 
    'Gotth.': 'Gotthold', 
    'Gottl.': 'Gottlob', 
    'Gust.': 'Gustav', 
    'Heinr.': 'Heinrich', 
    'Helm.': 'Helmut',
    'Herb.': 'Herbert', 
    'Herm.': 'Hermann', 
    'Hinr.': 'Hinrich', 
    'Hr.': 'Herman', 
    'Hrch.': 'Heinrich', 
    'Joh.': 'Johann', 
    'Johs.': 'Johannes', 
    'Jos.': 'Joseph', 
    'Jul.': 'Julius', 
    'Jürg.': 'Jürgen', 
    'Kasim.': 'Kasimir', 
    'Kl.': 'Karl', 
    'Klem.': 'Klemens', 
    'Konr.': 'Konrad', 
    'Konst.': 'Konstantin',
    'Korn.': 'Kornelius',
    'Kornel.': 'Kornelius',  
    'Ldw.': 'Ludwig', 
    'Leonh.': 'Leonhard', 
    'Loth.': 'Lothar',
    'Ludw.': 'Ludwig', 
    'Mart.': 'Martin', 
    'Matth.': 'Matthias', 
    'Mich.': 'Michael', 
    'Nik.': 'Nikolaus', 
    'Nikl.': 'Niklas',      ## ???
    'Nikol.': 'Nikolaus', 
    'Osk.': 'Oskar', 
    'Osw.': 'Oswald', 
    'Pet.': 'Peter',
    'Phil.': 'Phillipp', 
    'Reinh.': 'Reinhard', 
    'Rich.': 'Richard', 
    'Richa.': 'Richard', 
    'Rob.': 'Robert', 
    'Rud.': 'Rudolf', 
    'Rupr.': 'Ruprecht',
    'Sal.': 'Salomon', 
    'Sam.': 'Samuel', 
    'Sebast.': 'Sebastian',
    'Siegfr.': 'Siegfried', 
    'Sigism.': 'Sigismund', 
    'Siegm.': 'Sigmund',
    'Stanisl.': "Stanislaw",   ## Korrekte Schreibung?
    'Th.': 'Theodor', 
    'Theod.': 'Theodor', 
    'Thom.': 'Thomas',
    'Traug.': 'Traugott', 
    'Ulr.': 'Ulrich', 
    'Val.': 'Valentin',
    'Vikt.': 'Viktor', 
    'Volkm.': 'Volkmar', 
    'Waldem.': 'Waldemar', 
    'Walt.': 'Walter', 
    'Wilh.': 'Wilhelm', 
    'Wolfg.': 'Wolfgang', 
    'X.': 'Xaver',
    'Xav.': 'Xaver'
}

def to_raw(string):
    return fr"{string}"

for key in most_used_abrevations:
    ext_key = key.replace(".", "\.")
    df_cars._Vorname.replace(to_raw(ext_key), most_used_abrevations[key] + " ",
                             inplace=True, regex=True)

    # Ergänzung! Wir wollen die Namen mit einer Änderung noch markieren
    relevant_df = df_cars.Vorname.str.contains(ext_key, na=False, regex=True)
    df_cars.loc[relevant_df, "_Vorname_Adaption"] = True

<>:103: SyntaxWarning: invalid escape sequence '\.'
<>:103: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_17851/1874431846.py:103: SyntaxWarning: invalid escape sequence '\.'
  ext_key = key.replace(".", "\.")
/tmp/ipykernel_17851/1874431846.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cars._Vorname.replace(to_raw(ext_key), most_used_abrevations[key] + " ",
/tmp/ipykernel_17851/1874431846.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment us

In [66]:
df_cars[df_cars._Vorname_Adaption][["Familienname", "Vorname", "_Vorname"]].head(10)

,Familienname,Vorname,_Vorname
16,Reinhardt,Wilh.,Wilhelm
48,Günther,Wilh.,Wilhelm
57,"Binne, jr.",Wilh.,Wilhelm
65,Kuthe,Wilh.,Wilhelm
75,Hornschuh,Friedr.,Friedrich
80,Hornschuh,Fr.,Friedrich
84,Wachsmuth,Emil Edm.,Emil Edmund
119,Puttfarken,Wilh.,Wilhelm
127,Stucken,Fr.,Friedrich
128,Ernst,Friedr.,Friedrich


In [67]:
abbrevations = df_cars[(~df_cars._Vorname.isnull()) & (df_cars._Vorname.str.contains(r"\.", regex=True ))]._Vorname.unique()
len(abbrevations)

925

In [68]:
set(abbrevations)

{'A.',
 'A. A. K.',
 'A. C.',
 'A. C. F. O.',
 'A. D.',
 'A. F.',
 'A. F. H. K.',
 'A. Friedrich ',
 'A. H.',
 'A. H. A.',
 'A. H. C.',
 'A. J.',
 'A. J. Arno',
 'A. Joseph ',
 'A. L.',
 'A. L., A. G.',
 'A. M.',
 'A. M. A. A.',
 'A. O.',
 'A. O. R.',
 'A. P.',
 'A. Paul',
 'A. Richard ',
 'A. Robert',
 'A. Theodor ',
 'A. W.',
 'A. W. H.',
 'A. Wwe.',
 'A., jr.',
 'Abraham S.',
 'Adalb.',
 'Adolf  Albert  M.',
 'Adolf  Johann  M. G.',
 'Adolf  K Kathar.',
 'Adolf C.',
 'Adolf jr.',
 'Adolf, Rudolf u. Julius, Gebr.',
 'Adr.',
 'Afr.',
 'Al.',
 'Albert  Br.',
 'Albert  E.',
 'Albert  jr.',
 'Albert jr.',
 'Albert, jr. und Max',
 'Albr.',
 'Alexander  G.',
 'Alf.',
 'Alfred A.',
 'Alfred H.',
 'Alois jr.',
 'Alois, jr.',
 'Alwin, jr.',
 'Amand.',
 'Anast.',
 'Andr.',
 'Andr. Ph.',
 'Ang.',
 'Anton  Friedrich  H.',
 'Anton, jr.',
 'Arn.',
 'Art.',
 'Art. Sigismund ',
 'Artur F.',
 'Artur K. B.',
 'Artur u. Adolf',
 'Artur, jr.',
 'August  A.',
 'August  C.',
 'August  Friedrich  H.',
 'Au

In [62]:
df_cars.to_csv(cleaned_data_file, sep= "\t", index=False)  